<a href="https://colab.research.google.com/github/ryo-k2mt/lodging-price-prediction/blob/main/lodging_price_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# google drive へマウント
from google.colab import drive
drive.mount('/content/drive')

# ライブラリのインポート

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

# データの読み込み

# 前処理

# データの分割

# 学習

# 推論